In [ ]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86

In [ ]:
from google.colab import userdata

AWS_ACCESS_KEY_ID = userdata.get('AWS_ACCESS_KEY_ID')
AWS_DEFAULT_REGION = userdata.get('AWS_DEFAULT_REGION')
AWS_SECRET_ACCESS_KEY = userdata.get('AWS_SECRET_ACCESS_KEY')

In [ ]:
!aws configure set AWS_ACCESS_KEY_ID $AWS_ACCESS_KEY_ID
!aws configure set AWS_SECRET_ACCESS_KEY $AWS_SECRET_ACCESS_KEY
!aws configure set AWS_DEFAULT_REGION $AWS_DEFAULT_REGION

In [ ]:
import mlflow

In [ ]:
IP = "3.19.222.199"
PROTOCOL = "http"
PORT = "5000"

In [ ]:
TRACKING_URI = f"{PROTOCOL}://{IP}:{PORT}/"
EXPERIMENT_NAME = "EXP 4 - Handling Imbalanced Dataset"

In [ ]:
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

2025/10/13 22:30:16 INFO mlflow.tracking.fluent: Experiment with name 'EXP 4 - Handling Imbalanced Dataset' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://my-mlflow-bucket-1299/706433518517665041', creation_time=1760394616433, experiment_id='706433518517665041', last_update_time=1760394616433, lifecycle_stage='active', name='EXP 4 - Handling Imbalanced Dataset', tags={}>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
from mlflow.models import infer_signature
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

In [ ]:
df = pd.read_csv('dataset.csv')
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [ ]:
df = df.dropna()
df = df.drop_duplicates()

In [ ]:
def run_imbalanced_experiment(imbalanced_method):
  ngram_range = (1,3)
  max_features = 1_000

  X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)

  vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
  X_train_vec = vectorizer.fit_transform(X_train)
  X_test_vec = vectorizer.transform(X_test)

  if imbalanced_method == "class_weights":
    class_weight = 'balanced'
  else:
    class_weight = None

  #Resampling techniques

  if imbalanced_method == 'oversampling':
    smote = SMOTE(random_state=42)
    X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
  elif imbalanced_method == 'adasyn':
    adasyn = ADASYN(random_state= 42)
    X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
  elif imbalanced_method == 'undersampling':
    rus = RandomUnderSampler(random_state=42)
    X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
  elif imbalanced_method == 'smote_enn':
    smote_enn = SMOTEENN(random_state=42)
    X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

  with mlflow.start_run() as run:

    mlflow.set_tag("mlflow.runName",f"Imbalance_{imbalanced_method}_RandomForest_TFIDF_Trigram")
    mlflow.set_tag("experiment_type","imbalance_handling")
    mlflow.set_tag("model_type","RandomForestClassifier")

    #add description
    mlflow.set_tag("Description",f"RandomForest with TF-IDF Trigrams, imbalance handling method = {imbalanced_method}")

    #log vectorizer parameters
    mlflow.log_param("Vectorizer_type", "TF-IDF")
    mlflow.log_param("ngram_range", ngram_range)
    mlflow.log_param("vectorizer_max_features",max_features)

    #log random forest parameters
    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators",n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("imbalance method",imbalanced_method)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth= max_depth, random_state=42, class_weight=class_weight)
    model.fit(X_train_vec, y_train)

    y_pred = model.predict(X_test_vec)

    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy",accuracy)

    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
      if isinstance(metrics, dict):
        for metric, value in metrics.items():
          mlflow.log_metric(f"{label}_{metric}", value)

    #log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize = (8,6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Spectral')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance Method = {imbalanced_method}")
    file_name = f"confusion_matrix_{imbalanced_method}.png"
    plt.savefig(file_name)
    mlflow.log_artifact(file_name)
    plt.close()

    #log model
    mlflow.sklearn.log_model(model,
                             name = f"random_forest_model_tfidf_trigrams_{imbalanced_method}",
                             signature=infer_signature(X_test_vec, y_test)
                             )


In [ ]:
imbalance_methods = ['class_weights','oversampling','adasyn','undersampling','smote_enn']

for method in imbalance_methods:
  run_imbalanced_experiment(method)

/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigram at: http://3.19.222.199:5000/#/experiments/706433518517665041/runs/9dbfdcebac4c4c8abe715b0af03f0020
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/706433518517665041


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigram at: http://3.19.222.199:5000/#/experiments/706433518517665041/runs/9b2535ffe2ef4072bf09c53df0600883
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/706433518517665041


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigram at: http://3.19.222.199:5000/#/experiments/706433518517665041/runs/df3473f53dfd4aa2bfebbdf38ec79405
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/706433518517665041


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigram at: http://3.19.222.199:5000/#/experiments/706433518517665041/runs/c53a61d5c95e4f1f9ac3780106c75844
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/706433518517665041


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigram at: http://3.19.222.199:5000/#/experiments/706433518517665041/runs/5ff8fdaf5de945c6b1b74e3d8535e14e
🧪 View experiment at: http://3.19.222.199:5000/#/experiments/706433518517665041
